<a href="https://colab.research.google.com/github/YosyValero/jvalero59_TFG/blob/main/Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Baseline TFG

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
path_tsv = '/content/drive/MyDrive/Dataset/tsv/'

In [ ]:
df_dissea =  pd.read_csv(path_tsv + 'dissea.tsv', sep='\t', header=0).loc[:, ('tweets_id', 'type')].rename(columns={'tweets_id': 'tweet_id'})

In [ ]:
df_prof =  pd.read_csv(path_tsv + 'prof.tsv', sep='\t', header=0).loc[:, ('tweet_id', 'type')]

In [ ]:
df_prof.drop(df_prof[df_prof['type']=='-'].index, inplace=True)

In [ ]:
df_prof = df_prof.replace(['PROFESION', 'SITUACION_LABORAL', 'ACTIVIDAD', 'FIGURATIVA'], ['NO_ENFERMEDAD', 'NO_ENFERMEDAD', 'NO_ENFERMEDAD', 'NO_ENFERMEDAD'])

In [ ]:
df_prof_norep = df_prof.drop_duplicates(ignore_index=True)

In [ ]:
df_disse_norep = df_dissea.drop_duplicates(ignore_index=True)

In [ ]:
muestras_min = min([len(df_disse_norep), len(df_prof_norep)])

In [85]:
muestras = 260

In [86]:
if muestras_min < muestras:
  muestras = muestras_min

In [87]:
df_xamples_diss = df_disse_norep.head(muestras)

In [88]:
df_xamples_nodiss = df_prof_norep.head(muestras)

In [89]:
df_xamples_nodiss = df_xamples_nodiss.rename(index=lambda x: x + muestras)

In [90]:
df_tot = pd.concat([df_xamples_diss, df_xamples_nodiss])


In [ ]:
df_tot

In [92]:
df_tot.insert(2, 'content_tuit', value='')

In [93]:
def get_tuit(type_tuit, name_tuit):
  sub_path = path_tsv + 'train-valid-txt-files/'
  if type_tuit == 'ENFERMEDAD':
    path_tuit = sub_path + 'dissea'
  else:
    path_tuit = sub_path + 'prof'

  path_tuit = path_tuit + '/training/' + name_tuit + '.txt'
  tuit = open(path_tuit, mode='r', encoding='utf-8')
  content_tuit = tuit.read()
  tuit.close()

  return content_tuit

In [94]:
for i in range(len(df_tot)):
    name_tuit = df_tot.loc[i, 'tweet_id']
    type_tuit = df_tot.loc[i, 'type']
    df_tot.at[i, 'content_tuit'] = get_tuit(type_tuit, str(name_tuit))

In [95]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [96]:
symbols = [
    '¿','?','~','`','!','¡','@','$','%','^',
    '*','(',')','_','-','+','=','{','}','[',
    ']','\\',';','<','>','/', '.', ',','&','\n'
    ,'\r','\t', '|', '“', '"', '–', '”', '©', '-', '—',
    '…', ';', ':', '‘','’', '...', '#', '«', '»', '``',
    "''", '..', '...', '--'
]

In [97]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [98]:
def limpia_tuit_sp(text):
    str_limp = ""
    stp_wors_mas_sym = stopwords.words('spanish') + symbols

    # Eliminamos las urls de los tuits
    text = re.sub('http[s]*://[a-zA-Z0-9/.-]*|www.[a-zA-Z0-9/.-]*', '', text)

    # Eliminamos numeros
    text = re.sub('[0-9]*','', text)
    # print(text)

    # Eliminamos emoticonos
    text = remove_emoji(text)

    # Eliminamos otros caracteres
    text = re.sub('[¿,_,?,¡,!,-]*','', text)

    # Tokenizamos los tuits
    text_tok = word_tokenize(text)
    for t_tok in text_tok:
        if not t_tok in stp_wors_mas_sym:
            str_limp = str_limp + " " + t_tok
    str_limp =  str_limp.lower()
    return str_limp

In [99]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [100]:
x, y = np.asarray(df_tot['content_tuit']), np.asarray(df_tot['type'])
x_limpieza = [limpia_tuit_sp(t) for t in x]

In [ ]:
df_tot.tail(7)

In [ ]:
print(x_limpieza[116])

In [ ]:
print(df_tot['content_tuit'][116])

In [104]:
from sklearn.feature_extraction.text import CountVectorizer

In [105]:
onegram_vectorizer = CountVectorizer()

In [106]:
X = onegram_vectorizer.fit_transform(x_limpieza).toarray()

In [107]:
from sklearn import svm
from sklearn.model_selection import train_test_split

In [108]:
x_train, x_test, y_train, y_test = train_test_split(X, y)
clf = svm.SVC()

In [ ]:
clf.fit(x_train, y_train)

In [ ]:
clf.score(x_test, y_test)

In [ ]:
clf.set_params(kernel='rbf').fit(x_train, y_train)

In [ ]:
clf.score(x_test, y_test)